# Test pricing transition on localnet

This notebook connects to a running localnet node, inspects pricing transition behavior,
and compares current vs scheduled prices around transition end.

## Configure connection

In [1]:
Cookie = 'localnet',
Node = 'main-localnet@127.0.0.1',

HostHasDot =
    case string:split(atom_to_list(node()), "@") of
        [_Name, Host] ->
            case string:find(Host, ".") of
                nomatch ->
                    false;
                _ ->
                    true
            end;
        _ ->
            false
    end,

_ =
    case {node(), HostHasDot} of
        {nonode@nohost, _} ->
            net_kernel:start([list_to_atom("pricing_notebook@127.0.0.1"), longnames]);
        {_, true} ->
            ok;
        {_, false} ->
            net_kernel:stop(),
            net_kernel:start([list_to_atom("pricing_notebook@127.0.0.1"), longnames])
    end,

erlang:set_cookie(node(), Cookie).

true


In [2]:
%% RPC helpers and record accessors
%% The ierl kernel does not provide rr/c:rr, so compile a tiny accessor module.
AccessorModule =
    lists:flatten([
        "-module(nb_block_accessors).\n",
        "-export([price_per_gib_minute/1, scheduled_price_per_gib_minute/1, denomination/1,\n",
        "         height/1, kryder_plus_rate_multiplier/1]).\n",
        "-include_lib(\"arweave/include/ar.hrl\").\n",
        "price_per_gib_minute(B) -> B#block.price_per_gib_minute.\n",
        "scheduled_price_per_gib_minute(B) -> B#block.scheduled_price_per_gib_minute.\n",
        "denomination(B) -> B#block.denomination.\n",
        "height(B) -> B#block.height.\n",
        "kryder_plus_rate_multiplier(B) -> B#block.kryder_plus_rate_multiplier.\n"
    ]),
TmpDir = ".tmp/notebooks/",
filelib:ensure_dir(TmpDir),
AccessorPath = filename:join([TmpDir, "nb_block_accessors.erl"]),
ok = file:write_file(AccessorPath, AccessorModule),
{ok, nb_block_accessors, AccessorBin} = compile:file(AccessorPath, [binary]),
{module, nb_block_accessors} = code:load_binary(nb_block_accessors, AccessorPath, AccessorBin),

PricePerGiBMinute =
    fun(B) ->
        nb_block_accessors:price_per_gib_minute(B)
    end,
ScheduledPricePerGiBMinute =
    fun(B) ->
        nb_block_accessors:scheduled_price_per_gib_minute(B)
    end,
Denomination =
    fun(B) ->
        nb_block_accessors:denomination(B)
    end,
Height =
    fun(B) ->
        nb_block_accessors:height(B)
    end,
KryderPlusRateMultiplier =
    fun(B) ->
        nb_block_accessors:kryder_plus_rate_multiplier(B)
    end,

RPCCall =
    fun(Module, Function, Args) ->
        rpc:call(Node, Module, Function, Args)
    end,

RPCNodeGetHeight =
    fun() ->
        RPCCall(ar_node, get_height, [])
    end,

RPCPricingTransitionStart =
    fun() ->
        RPCCall(ar_pricing_transition, transition_start_2_7_2, [])
    end,

RPCPricingTransitionLength =
    fun() ->
        RPCCall(ar_pricing_transition, transition_length_2_7_2, [])
    end,

RPCBlockIndexGetByHeight =
    fun(HeightValue) ->
        RPCCall(ar_block_index, get_element_by_height, [HeightValue])
    end,

RPCBlockGet =
    fun(BlockHash) ->
        RPCCall(ar_storage, read_block, [BlockHash])
    end,

GetFields =
    fun(Block) ->
        element(1, ar_serialize:block_to_json_struct(Block))
    end,

ToInt =
    fun(Value) ->
        case Value of
            V when is_integer(V) ->
                V;
            V when is_binary(V) ->
                binary_to_integer(V);
            V when is_list(V) ->
                list_to_integer(V);
            V when is_float(V) ->
                trunc(V);
            _ ->
                Value
        end
    end,

ok.

ok


In [3]:
{net_adm:names('127.0.0.1'), node(), erlang:get_cookie()}.

{{ok,[{"erlang_1ePN0QVP",51755},
      {"main-localnet",55436},
      {"pricing_notebook",59029}]},
 'pricing_notebook@127.0.0.1',localnet}


## Connectivity check

In [4]:
{Node, net_adm:ping(Node)}.

{'main-localnet@127.0.0.1',pong}


In [5]:
RPCNodeGetHeight().

1818163


## Compute transition window

In [6]:
TransitionStart = rpc:call(Node, ar_pricing_transition, transition_start_2_7_2, []),
TransitionLength = rpc:call(Node, ar_pricing_transition, transition_length_2_7_2, []),
TransitionEnd = TransitionStart + TransitionLength,
StartHeight = TransitionEnd - 100,
StopHeight = TransitionEnd + 100,

#{
    transition_start => TransitionStart,
    transition_end => TransitionEnd,
    start_height => StartHeight,
    stop_height => StopHeight
}.

## Mine blocks around the transition end

In [ ]:
CurrentHeight = rpc:call(Node, ar_node, get_height, []),
case CurrentHeight < StopHeight of
    true ->
        ok = rpc:call(Node, ar_localnet, mine_until_height, [StopHeight]),
        rpc:call(Node, ar_node, get_height, []);
    false ->
        CurrentHeight
end.

## Fetch prices around transition end

In [ ]:
GetBlock =
    fun(Height) ->
        BlockHash = rpc:call(Node, ar_block_index, get_element_by_height, [Height]),
        case rpc:call(Node, ar_block_cache, get, [block_cache, BlockHash]) of
            not_found ->
                {error, {block_not_found, Height}};
            Block ->
                Block
        end
    end,

ToInt =
    fun(Value) ->
        case Value of
            Bin when is_binary(Bin) ->
                binary_to_integer(Bin);
            Int when is_integer(Int) ->
                Int;
            _ ->
                undefined
        end
    end,

GetFields =
    fun(Block) ->
        {Fields} = rpc:call(Node, ar_serialize, block_to_json_struct, [Block]),
        Fields
    end,

GetPriceInfo =
    fun(Height) ->
        case GetBlock(Height) of
            {error, _} = Error ->
                #{height => Height, error => Error};
            Block ->
                Fields = GetFields(Block),
                Price = ToInt(proplists:get_value(price_per_gib_minute, Fields)),
                Scheduled = ToInt(proplists:get_value(scheduled_price_per_gib_minute, Fields)),
                Denomination = ToInt(proplists:get_value(denomination, Fields)),
                #{height => Height, price => Price, scheduled_price => Scheduled,
                    denomination => Denomination}
        end
    end,

Heights = lists:seq(StartHeight, StopHeight),
Prices = [GetPriceInfo(Height) || Height <- Heights],
Prices.

## Validate price fields via recalc

In [ ]:
ValidateHeight =
    fun(Height) ->
        case {GetBlock(Height - 1), GetBlock(Height)} of
            {{error, _} = Error, _} ->
                {Height, Error};
            {_, {error, _} = Error} ->
                {Height, Error};
            {PrevBlock, Block} ->
                PrevFields = GetFields(PrevBlock),
                CurrFields = GetFields(Block),
                PrevDenomination = ToInt(proplists:get_value(denomination, PrevFields)),
                Denomination = ToInt(proplists:get_value(denomination, CurrFields)),
                {ExpectedPrice, ExpectedScheduled} =
                    rpc:call(Node, ar_pricing, recalculate_price_per_gib_minute, [PrevBlock]),
                ExpectedPrice2 =
                    rpc:call(Node, ar_pricing, redenominate,
                        [ExpectedPrice, PrevDenomination, Denomination]),
                ExpectedScheduled2 =
                    rpc:call(Node, ar_pricing, redenominate,
                        [ExpectedScheduled, PrevDenomination, Denomination]),
                Price = ToInt(proplists:get_value(price_per_gib_minute, CurrFields)),
                Scheduled = ToInt(proplists:get_value(scheduled_price_per_gib_minute, CurrFields)),
                case {ExpectedPrice2 == Price, ExpectedScheduled2 == Scheduled} of
                    {true, true} ->
                        ok;
                    _ ->
                        {Height, {price_mismatch, #{expected => ExpectedPrice2,
                            actual => Price, expected_scheduled => ExpectedScheduled2,
                            actual_scheduled => Scheduled}}}
                end
        end
    end,

Checks = [ValidateHeight(Height) || Height <- lists:seq(StartHeight + 1, StopHeight)],
Failures = [Check || Check <- Checks, Check =/= ok],
case Failures of
    [] ->
        ok;
    _ ->
        Failures
end.

## Validate v2 pricing post-transition

In [ ]:
ValidateV2 =
    fun(Height) ->
        case rpc:call(Node, ar_pricing_transition, is_v2_pricing_height, [Height]) of
            true ->
                case GetBlock(Height) of
                    {error, _} = Error ->
                        {Height, Error};
                    Block ->
                        Fields = GetFields(Block),
                        Price = ToInt(proplists:get_value(price_per_gib_minute, Fields)),
                        V2Price = rpc:call(Node, ar_pricing, get_v2_price_per_gib_minute,
                            [Height, Block]),
                        case Price == V2Price of
                            true ->
                                ok;
                            false ->
                                {Height, {v2_price_mismatch, #{expected => V2Price, actual => Price}}}
                        end
                end;
            false ->
                ok
        end
    end,

V2Checks = [ValidateV2(Height) || Height <- lists:seq(StartHeight, StopHeight)],
V2Failures = [Check || Check <- V2Checks, Check =/= ok],
case V2Failures of
    [] ->
        ok;
    _ ->
        V2Failures
end.

## Plot prices